![](https://raw.githubusercontent.com/tqdm/tqdm/master/images/logo.gif)

# tqdm 

`tqdm` means "progress" in Arabic (taqadum, تقدّم) and is an abbreviation for "I love you so much" in Spanish (te quiero demasiado).

![](https://raw.githubusercontent.com/tqdm/tqdm/master/images/tqdm.gif)

---

## Usage

`tqdm` is very versatile and can be used in a number of ways. The three main ones are given below.

### Iterable-based

Wrap `tqdm()` around any iterable:

In [14]:
from tqdm import tqdm

In [15]:
import time

text = ""
for char in tqdm(["a", "b", "c", "d"]):
    time.sleep(0.25)
    text = text + char

100%|██████████| 4/4 [00:01<00:00,  3.93it/s]


`trange(i)` is a special optimised instance of `tqdm(range(i))`:

In [16]:
from tqdm import trange

for i in trange(1_000_000):
    pass

100%|██████████| 1000000/1000000 [00:00<00:00, 1849858.32it/s]


Instantiation outside of the loop allows for manual control over `tqdm()`:

In [17]:
pbar = tqdm(["a", "b", "c", "d"])
for char in pbar:
    time.sleep(0.25)
    pbar.set_description("Processing %s" % char)

Processing d: 100%|██████████| 4/4 [00:01<00:00,  3.90it/s]


### Manual

Manual control on `tqdm()` updates by using a with statement:

In [18]:
with tqdm(total=100) as pbar:
    for i in range(10):
        time.sleep(0.25)
        pbar.update(10)

100%|██████████| 100/100 [00:02<00:00, 39.11it/s]


If the optional variable total (or an iterable with `len()`) is provided, predictive stats are displayed.

`with` is also optional (you can just assign `tqdm()` to a variable, but in this case don't forget to `del` or `close()` at the end:

In [19]:
pbar = tqdm(total=100)
for i in range(10):
    time.sleep(0.25)
    pbar.update(10)
pbar.close()

100%|██████████| 100/100 [00:02<00:00, 39.24it/s]


It can also be executed as a module with pipes:

In [20]:
! seq 999999 | tqdm --unit_scale | wc -l

1.00Mit [00:00, 1.17Mit/s]
  999999


## Advance usage

### Description and additional stats

Custom information can be displayed and updated dynamically on `tqdm` bars with the `desc` and `postfix` arguments:

In [21]:
from tqdm import trange
from random import random, randint
from time import sleep

with trange(100) as t:
    for i in t:
        # Description will be displayed on the left
        t.set_description('GEN %i' % i)
        # Postfix will be displayed on the right,
        # formatted automatically based on argument's datatype
        t.set_postfix(loss=random(), gen=randint(1,999), str='h',
                      lst=[1, 2])
        sleep(0.1)

with tqdm(total=10, bar_format="{postfix[0]} {postfix[1][value]:>8.2g}",
          postfix=["Batch", dict(value=0)]) as t:
    for i in range(10):
        sleep(0.1)
        t.postfix[1]["value"] = i / 2
        t.update()

GEN 99: 100%|██████████| 100/100 [00:11<00:00,  8.98it/s, gen=406, loss=0.256, lst=[1, 2], str=h]
Batch      4.5


### Nested progress bars

`tqdm` supports nested progress bars. Here's an example:

In [22]:
from tqdm import trange
from time import sleep

for i in trange(3, desc='1st loop'):
    for j in trange(50, desc='2nd loop', leave=False):
        sleep(0.01)

1st loop: 100%|██████████| 3/3 [00:01<00:00,  1.67it/s]


### Pandas Integration

Due to popular demand we've added support for `pandas` -- here's an example for `DataFrame.progress_apply` and `DataFrameGroupBy.progress_apply`:

In [24]:
import pandas as pd
import numpy as np
from tqdm import tqdm

df = pd.DataFrame(np.random.randint(0, 100, (100_00, 6)))

# Register `pandas.progress_apply` and `pandas.Series.map_apply` with `tqdm`
# (can use `tqdm_gui`, `tqdm_notebook`, optional kwargs, etc.)
tqdm.pandas(desc="my bar!")

# Now you can use `progress_apply` instead of `apply`
# and `progress_map` instead of `map`
df.progress_apply(lambda x: x**2)
# can also groupby:
# df.groupby(0).progress_apply(lambda x: x**2)

my bar!: 100%|██████████| 6/6 [00:00<00:00, 411.54it/s]


,0,1,2,3,4,5
0,961,64,1764,5776,400,6889
1,6084,1444,625,529,4624,4225
2,5625,9216,6084,961,8100,5476
3,7744,2304,6561,7396,400,8100
4,2025,3969,1,1156,1681,196
5,3136,1764,16,49,9216,25
6,6889,7569,7225,5041,400,361
7,64,144,6241,1681,6561,3364
8,4761,1764,9604,8836,1296,2809
9,2601,289,484,2209,1024,1849


### Writing messages

Since `tqdm` uses a simple printing mechanism to display progress bars, you should not write any message in the terminal using `print()` while a progressbar is open.

To write messages in the terminal without any collision with `tqdm` bar display, a `.write()` method is provided:

In [25]:
from tqdm import tqdm, trange
from time import sleep

bar = trange(10)
for i in bar:
    # Print using tqdm class method .write()
    sleep(0.1)
    if not (i % 3):
        tqdm.write("Done task %i" % i)
    # Can also use bar.write()

 20%|██        | 2/10 [00:00<00:00,  9.40it/s]

Done task 0


 50%|█████     | 5/10 [00:00<00:00,  9.50it/s]

Done task 3


 80%|████████  | 8/10 [00:00<00:00,  9.49it/s]

Done task 6


100%|██████████| 10/10 [00:01<00:00,  9.42it/s]

Done task 9


---

## Do your own experiments here 👇

Try `tqdm` youself by adding your code below and running your own experiments.

In [ ]:
import tqdm

# your code here
tqdm.